In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd

# Munging US CPC to IPC into data Checking Compatibility

## SQL Query used on PATSTAT 2015 Spring

## Importing
[to top](#Table-of-Contents) 

In [ ]:
pwd

# CPC

In [ ]:
f_all_grants = '../Data_Play/all_grant_only_prior_US_cpc_v1.csv'

In [ ]:
USdf = pd.read_csv(f_all_grants, engine='c', dtype={'publn_nr':str,'appln_id':str})
#dont need ID's

# IPC

In [ ]:
tot_file = '../Data_Play/all_grant_only_prior_v1.csv'

In [ ]:
df = pd.read_csv(tot_file, engine='c', dtype={'publn_nr':str,'appln_id':str})
#dont need ID's

In [ ]:
USdf_needed = USdf[~USdf.appln_id.isin(df.appln_id)]; USdf_needed.head()

In [ ]:
print len(USdf_needed.appln_id.value_counts())
# num of extra patents getting form CPC

In [ ]:
print USdf_needed.cpc_class_symbol[~USdf_needed.cpc_class_symbol.str.contains(' ')].map(len).value_counts()
print USdf_needed.cpc_class_symbol.map(len).value_counts() 
# CPC classifcaitons always to group level or beyond!!

#### Plan B (checking): 
- Rip id's from each df
- Drop duplicates from these serieses
- remove the IPC ids from the CPC list
- use this list to rip only the need appln id codes from USdf
- concate this onto df

# Checking compatibility

In [ ]:
# make messy copies
df_c = df.copy()
USdf_n_c = USdf_needed.copy()

In [ ]:
# truncate on '/'
df_c.ipc_class_symbol = df_c.ipc_class_symbol.map(lambda x: x.split('/')[0])
USdf_n_c.cpc_class_symbol = USdf_n_c.cpc_class_symbol.map(lambda x: x.split('/')[0])
print df_c.head()
print USdf_n_c.head()
print '\n'
print df_c.ipc_class_symbol.map(len).value_counts()
print USdf_n_c.cpc_class_symbol.map(len).value_counts()

In [ ]:
# df_c into frame of only groups and only subclasses by filtering on ipc contain whitespace
df_ipc_prime = df_c[df_c.ipc_class_symbol.str.contains(' ')]
print df_ipc_prime
df_ipc_4 = df_c[~df_c.ipc_class_symbol.str.contains(' ')]
print df_ipc_4
print '\n'
print df_ipc_prime.ipc_class_symbol.map(len).value_counts()
print df_ipc_4.ipc_class_symbol.map(len).value_counts()
# checks all good

In [ ]:
c = 0
for code in df_ipc_prime.ipc_class_symbol:
    if code[5] != ' ':
        c = c+1
print c

In [ ]:
# Split CPCs into munged need and not
USdf_cpc_spaces = USdf_n_c[USdf_n_c.cpc_class_symbol.str.contains(' ')]
print USdf_cpc_spaces.head()
print len(USdf_cpc_spaces)
print '\n'
USdf_cpc_only = USdf_n_c[~USdf_n_c.cpc_class_symbol.str.contains(' ')]
print USdf_cpc_only.head()
print len(USdf_cpc_only)

In [ ]:
# Check the not mungeneed all exist in ipc, should get no results if all matching
USdf_cpc_spaces_no_match = USdf_cpc_spaces[~USdf_cpc_spaces.cpc_class_symbol.isin(df_ipc_prime.ipc_class_symbol)]
USdf_cpc_spaces_with_match = USdf_cpc_spaces[USdf_cpc_spaces.cpc_class_symbol.isin(df_ipc_prime.ipc_class_symbol)]
print len(USdf_cpc_spaces_no_match)
print '\n'
#print USdf_cpc_spaces_no_match.cpc_class_symbol.drop_duplicates()

print len(USdf_cpc_spaces_with_match)

In [ ]:
# make function to munge the CPC only in ipc
def try_cpc_to_ipc(code):
    code = code[:4] + '  ' + code[6:]
    if code[6] == '0':
        code = code[:6] + ' ' + code[7:]
    return code

## Cannot be reliably munged
See CPC guide page 8 for examples of ambiguous codes

## CPC to IPC as far is it could go
## Now make final data sets

In [ ]:
US_extra_fin =  USdf_cpc_spaces_with_match.copy()
US_extra_fin.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True)
US_extra_fin

In [ ]:
# append the prime IPC with the prime US, making final data set for usage
df_final_still_EP = df_ipc_prime.append(US_extra_fin, ignore_index=True)
print df_final_still_EP
print len(US_extra_fin)+len(df_ipc_prime)

In [ ]:
# 46530055

In [ ]:
df_final_still_EP

In [ ]:
#pk.dump(df_final_still_EP, open('pickles_and_munged_csv/final_data_still_EP_v1.pk', 'wb'))

## Now look at fraction of patents that have a missing code per auth per year

In [ ]:
# Make set for no filter on code consistency
full_no_filter_df = df.append(USdf_needed.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True),
                              ignore_index=True)

In [ ]:
who 

In [ ]:
# Make set of rejected codes
reject_codes = df_ipc_4.append([USdf_cpc_only.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True),
                               USdf_cpc_spaces_no_match.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True)],
                              ignore_index=True)

In [ ]:
who

In [ ]:
print len(df_final_still_EP)
print len(full_no_filter_df)
print len(reject_codes)